In [3]:
import os
import logging
from pathlib import Path

import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm
import sys
from joblib import Parallel, delayed
logging.basicConfig(level=logging.ERROR)


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [13]:
# 
class CFG:
    def __init__(self, mode="train", kaggle_notebook=False, debug=False):
        assert mode in ["train", "inference"], "mode must be 'train' or 'inference'"
        self.mode = mode
        self.KAGGLE_NOTEBOOK = kaggle_notebook
        self.debug = debug

        # ===== Path Settings =====
        if self.KAGGLE_NOTEBOOK:
            self.OUTPUT_DIR = ''
            self.train_datadir = '/kaggle/input/birdclef-2025/train_audio'
            self.train_csv = '/kaggle/input/birdclef-2025/train.csv'
            self.test_soundscapes = '/kaggle/input/birdclef-2025/test_soundscapes'
            self.submission_csv = '/kaggle/input/birdclef-2025/sample_submission.csv'
            self.taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv'
            self.spectrogram_npy = '/kaggle/input/birdclef25-mel-spectrograms/birdclef2025_melspec_5sec_256_256.npy'
            self.model_path = "/kaggle/input/birdclef-2025-baseline-fold0-0404"
            
            self.device = "cpu"
            self.batch_size = 8
            self.n_jobs = 3
            
        else:
            self.OUTPUT_DIR = '../data/result/'
            self.train_datadir = '../data/raw/train_audio/'
            self.train_csv = '../data/raw/train.csv'
            self.test_soundscapes = '../data/raw/test_soundscapes/'
            self.submission_csv = '../data/raw/sample_submission.csv'
            self.taxonomy_csv = '../data/raw/taxonomy.csv'
            self.spectrogram_npy = '../data/processed/mel-spec_0329/birdclef2025_melspec_5sec_256_256.npy'
            self.MODELS_DIR = "../models/"
            self.model_path =  "../models/baseline_fold0_0404_vino/"
            
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
            self.batch_size = 32
            self.n_jobs = 3

        # ===== Model Settings =====
        self.model_name = 'efficientnet_b0'
        self.pretrained = True if mode == "train" else False
        self.in_channels = 1

        # ===== Audio Settings =====
        self.FS = 32000
        self.WINDOW_SIZE = 5
        self.TARGET_DURATION = 5
        self.TARGET_SHAPE = (256, 256)
        self.N_FFT = 1024
        self.HOP_LENGTH = 512
        self.N_MELS = 128
        self.FMIN = 50
        self.FMAX = 14000
        
        self.seed = 42
        
        self.smooth_center_weight = 0.6
        self.smooth_neighbor_weight = 0.2
        
        # ===== Inference Mode =====
        if mode == "inference":
            self.use_tta = False
            self.tta_count = 3
            self.threshold = 0.5

            self.use_specific_folds = False
            self.folds = [0, 1, 2, 3, 4]  # Used only if use_specific_folds is True

            self.debug_count = 3
            
            
    def update_debug_settings(self):
        if self.debug:
            self.epochs = 2
            self.selected_folds = [0]

In [14]:
"TODO: configを2つにわけるべきかも．柔軟に変える方はnotebook側で，固定したい方はmodule側とか"
"TODO: Debugモードになっていたらsubmissionでエラーになる"
"TODO: 疑似ラベル作成のためなら，事前にデータセットを準備するべき"
cfg = CFG(mode='inference', kaggle_notebook=False)

if cfg.KAGGLE_NOTEBOOK:
    !pip install -U openvino-telemetry  --no-index --find-links /kaggle/input/pip-hub
    !pip install -U openvino  --no-index --find-links /kaggle/input/pip-hub
    sys.path.append("/kaggle/input/birdclef-2025-libs/")
    
from openvino.runtime import Core
from module import models_lib, utils_lib, preprocess_lib, inference_lib

# Set seed
utils_lib.set_seed(cfg.seed)

In [15]:
print(f"Using device: {cfg.device}")
print(f"Loading taxonomy data...")
taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
species_ids = taxonomy_df['primary_label'].tolist()
num_classes = len(species_ids)
print(f"Number of classes: {num_classes}")

Using device: cuda
Loading taxonomy data...
Number of classes: 206


In [16]:
def process_audio_file(audio_path, cfg):
    """1ファイル分のmelspecデータを返す（row_id, melspecのリスト）"""
    dataset = []
    soundscape_id = Path(audio_path).stem
    try:
        audio_data, _ = librosa.load(audio_path, sr=cfg.FS)
        total_segments = int(len(audio_data) / (cfg.FS * cfg.WINDOW_SIZE))

        for segment_idx in range(total_segments):
            start = int(segment_idx * cfg.FS * cfg.WINDOW_SIZE)
            end = int(start + cfg.FS * cfg.WINDOW_SIZE)
            segment_audio = audio_data[start:end]

            mel_spec = preprocess_lib.process_audio_segment(segment_audio, cfg)
            row_id = f"{soundscape_id}_{(segment_idx + 1) * cfg.WINDOW_SIZE}"

            dataset.append({
                "row_id": row_id,
                "mel_spec": mel_spec
            })
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
    return dataset

def generate_melspec_dataset(cfg):
    test_dir = Path(cfg.test_soundscapes)
    if not test_dir.exists():
        print(f"Test directory {test_dir} does not exist.")
        return []

    test_files = list(test_dir.glob('*.ogg'))
    if len(test_files) == 0:
        print("No test audio files found.")
        return []

    if cfg.debug:
        print(f"Debug mode enabled, using only {cfg.debug_count} files")
        test_files = test_files[:cfg.debug_count]

    results = Parallel(n_jobs=cfg.n_jobs)(
        delayed(process_audio_file)(path, cfg) for path in tqdm(test_files, desc="Parallel melspec gen")
    )
    dataset = [item for sublist in results for item in sublist]
    return dataset

In [17]:
def load_openvino_models(vino_dir, cfg):
    models = []
    vino_dir = Path(vino_dir)
    xml_files = sorted(vino_dir.glob("model_fold*.xml"))

    for xml_path in xml_files:
        bin_path = xml_path.with_suffix(".bin")

        if not xml_path.exists() or not bin_path.exists():
            print(f"Warning: Missing files for {xml_path.stem}")
            continue

        core = Core()
        model_ir = core.read_model(xml_path)
        compiled_model = core.compile_model(model_ir, device_name="CPU")
        models.append(compiled_model)

    print(f"✅ Loaded {len(models)} OpenVINO models from {vino_dir}")
    return models


In [18]:
def load_openvino_models(vino_dir, cfg):
    models = []
    vino_dir = Path(vino_dir)
    xml_files = sorted(vino_dir.glob("model_fold*.xml"))

    for xml_path in xml_files:
        bin_path = xml_path.with_suffix(".bin")

        if not xml_path.exists() or not bin_path.exists():
            print(f"Warning: Missing files for {xml_path.stem}")
            continue

        core = Core()
        model_ir = core.read_model(xml_path)
        compiled_model = core.compile_model(model_ir, device_name="CPU")
        models.append(compiled_model)

    print(f"✅ Loaded {len(models)} OpenVINO models from {vino_dir}")
    return models

# === NEW ===
def run_inference_openvino(dataset, models_ir, cfg, species_ids):
    row_ids = []
    all_preds = []

    for i in range(0, len(dataset), cfg.batch_size):
        batch = dataset[i:i+cfg.batch_size]

        mel_list = [item["mel_spec"] for item in batch]
        input_tensor = np.stack(mel_list).astype(np.float32)  # (B, H, W)
        input_tensor = np.expand_dims(input_tensor, axis=1)  # (B, 1, H, W)

        preds_per_model = []
        for model in models_ir:
            input_layer = model.input(0)
            output_layer = model.output(0)
            result = model([input_tensor])[output_layer]
            probs = 1 / (1 + np.exp(-result))  # sigmoid
            preds_per_model.append(probs)

        avg_preds = np.mean(preds_per_model, axis=0)
        all_preds.append(avg_preds)
        row_ids.extend([item["row_id"] for item in batch])

    predictions = np.concatenate(all_preds, axis=0)
    return row_ids, predictions

In [19]:
def smooth_submission_df(submission_df, cfg):
    """
    Smooth predictions in a submission DataFrame and return the modified DataFrame.

    Each row's prediction is averaged with its neighbors using weights from cfg.

    :param submission_df: DataFrame containing submission data (row_id + prediction columns)
    :param cfg: Configuration object containing smoothing weights
    :return: Smoothed submission DataFrame
    """
    print("Smoothing submission predictions...")
    sub = submission_df.copy()
    cols = sub.columns[1:]
    groups = sub['row_id'].str.rsplit('_', n=1).str[0].values
    unique_groups = np.unique(groups)

    for group in unique_groups:
        idx = np.where(groups == group)[0]
        sub_group = sub.iloc[idx].copy()
        predictions = sub_group[cols].values
        new_predictions = predictions.copy()

        if predictions.shape[0] > 1:
            w_c = cfg.smooth_center_weight
            w_n = cfg.smooth_neighbor_weight
            new_predictions[0] = predictions[0] * (1 - w_n) + predictions[1] * w_n
            new_predictions[-1] = predictions[-1] * (1 - w_n) + predictions[-2] * w_n
            for i in range(1, predictions.shape[0] - 1):
                new_predictions[i] = (
                    predictions[i - 1] * w_n +
                    predictions[i] * w_c +
                    predictions[i + 1] * w_n
                )
        sub.iloc[idx, 1:] = new_predictions

    return sub

In [20]:
print("Generating dataset...")
dataset = generate_melspec_dataset(cfg)

print("Loading OpenVINO models...")
vino_dir = Path(cfg.model_path).with_name(Path(cfg.model_path).name)
models_ir = load_openvino_models(vino_dir, cfg)

if not models_ir:
    raise RuntimeError("No OpenVINO models found.")

print("Running OpenVINO inference...")
if len(dataset) > 0:
    row_ids, predictions = run_inference_openvino(dataset, models_ir, cfg, species_ids)
else:
    print("No test data available, generating empty submission.")
    row_ids = []
    predictions = []

submission_df = utils_lib.create_submission(row_ids, predictions, species_ids, cfg)
submission_path = os.path.join(cfg.OUTPUT_DIR, 'submission.csv')
submission_df.to_csv(submission_path, index=False)

print(f"Submission saved to {submission_path}")


Generating dataset...


/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use th

Parallel melspec gen:   0%|          | 0/71 [00:00<?, ?it/s]

Loading OpenVINO models...
✅ Loaded 1 OpenVINO models from ../models/baseline_fold0_0404_vino
Running OpenVINO inference...
Creating submission dataframe...
Submission saved to ../data/result/submission.csv


In [21]:
# Smooth and save separately
smoothed_df = smooth_submission_df(submission_df, cfg)
smoothed_submission_path = os.path.join(cfg.OUTPUT_DIR, 'submission_smoothed.csv')
smoothed_df.to_csv(smoothed_submission_path, index=False)
print(f"Smoothed submission saved to {smoothed_submission_path}")

Smoothing submission predictions...
Smoothed submission saved to ../data/result/submission_smoothed.csv


In [22]:
# 提出用ファイルを読み込む
submission = pd.read_csv(os.path.join(cfg.OUTPUT_DIR, 'submission.csv'))
submission.head()

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230420_074000_5,0.000048,0.000051,0.000346,0.000080,0.000710,0.000473,0.000346,0.000031,0.000015,...,0.009126,0.000804,0.000191,0.002323,0.000011,0.011687,0.000158,0.000779,0.000417,0.000315
1,H02_20230420_074000_10,0.000045,0.000031,0.000346,0.000261,0.000970,0.000444,0.000488,0.000051,0.000026,...,0.005911,0.001134,0.000245,0.000536,0.000005,0.000940,0.000070,0.002551,0.000203,0.001501
2,H02_20230420_074000_15,0.000058,0.000058,0.001245,0.000029,0.005730,0.001325,0.012821,0.000055,0.000048,...,0.025565,0.002801,0.000473,0.000607,0.000045,0.003707,0.000169,0.005555,0.000102,0.000055
3,H02_20230420_074000_20,0.000066,0.000085,0.000710,0.000048,0.002183,0.002116,0.011687,0.000055,0.000038,...,0.056652,0.007121,0.000570,0.002323,0.000014,0.015906,0.000261,0.024798,0.002715,0.000856
4,H02_20230420_074000_25,0.000038,0.000045,0.000444,0.000085,0.001284,0.000732,0.001099,0.000045,0.000020,...,0.005730,0.003075,0.000116,0.003173,0.000004,0.002551,0.000131,0.016403,0.000710,0.000217


In [23]:
submission = pd.read_csv(os.path.join(cfg.OUTPUT_DIR, 'submission_smoothed.csv'))
submission.head()

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230420_074000_5,0.000048,0.000047,0.000346,0.000116,0.000762,0.000467,0.000374,0.000035,0.000017,...,0.008483,0.000870,0.000202,0.001966,0.000010,0.009538,0.000141,0.001134,0.000375,0.000552
1,H02_20230420_074000_10,0.000049,0.000041,0.000526,0.000179,0.001870,0.000626,0.002926,0.000048,0.000028,...,0.010485,0.001401,0.000280,0.000908,0.000014,0.003643,0.000108,0.002797,0.000226,0.000975
2,H02_20230420_074000_15,0.000057,0.000058,0.000958,0.000079,0.004069,0.001307,0.010128,0.000054,0.000042,...,0.027852,0.003332,0.000447,0.000936,0.000031,0.005594,0.000167,0.008803,0.000645,0.000504
3,H02_20230420_074000_20,0.000059,0.000072,0.000764,0.000052,0.002713,0.001681,0.009796,0.000053,0.000036,...,0.040251,0.005448,0.000460,0.002150,0.000018,0.010795,0.000217,0.019270,0.001791,0.000568
4,H02_20230420_074000_25,0.000054,0.000067,0.000523,0.000094,0.001441,0.001004,0.003064,0.000063,0.000035,...,0.016381,0.003395,0.000201,0.002506,0.000007,0.005663,0.000183,0.014909,0.001012,0.000550


In [40]:
submission

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230420_074000_5,0.000252,0.000130,0.000480,0.000309,0.000906,0.002329,0.001198,0.000155,0.000109,...,0.011560,0.003790,0.005031,0.004018,0.000281,0.006332,0.000613,0.013817,0.005652,0.003098
1,H02_20230420_074000_10,0.000148,0.000061,0.000347,0.000384,0.001204,0.002448,0.000656,0.000091,0.000065,...,0.005254,0.004025,0.001697,0.002500,0.000127,0.004356,0.000214,0.024687,0.001512,0.002109
2,H02_20230420_074000_15,0.000277,0.000176,0.000610,0.000196,0.001639,0.001845,0.003718,0.000163,0.000178,...,0.014551,0.005425,0.002507,0.001302,0.000583,0.003832,0.000755,0.008711,0.002236,0.000753
3,H02_20230420_074000_20,0.000194,0.000085,0.000632,0.000080,0.002758,0.002911,0.007287,0.000103,0.000092,...,0.015231,0.007367,0.003911,0.002451,0.000901,0.004295,0.000706,0.025596,0.002079,0.000537
4,H02_20230420_074000_25,0.000129,0.000066,0.000305,0.000128,0.000664,0.001427,0.000510,0.000093,0.000067,...,0.011911,0.004681,0.001599,0.005021,0.000236,0.003576,0.000428,0.016568,0.004155,0.001141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,H02_20230502_080500_40,0.000030,0.000032,0.000007,0.000027,0.000018,0.000033,0.000033,0.000037,0.000048,...,0.000953,0.009681,0.000184,0.002661,0.000529,0.000439,0.004886,0.000504,0.001188,0.000384
848,H02_20230502_080500_45,0.000043,0.000049,0.000024,0.000009,0.000017,0.000071,0.000062,0.000079,0.000032,...,0.001438,0.019206,0.001147,0.005418,0.000578,0.004902,0.003024,0.002959,0.016983,0.001932
849,H02_20230502_080500_50,0.000011,0.000010,0.000012,0.000002,0.000008,0.000039,0.000037,0.000014,0.000004,...,0.009107,0.005016,0.000353,0.000682,0.001027,0.000997,0.006813,0.004871,0.002842,0.000279
850,H02_20230502_080500_55,0.000023,0.000027,0.000005,0.000017,0.000007,0.000016,0.000019,0.000023,0.000027,...,0.000973,0.006297,0.000053,0.001926,0.000868,0.000371,0.093312,0.000469,0.002867,0.000394


In [42]:
print("✅ Shape:", submission.shape)
print("✅ Columns:", submission.columns.tolist())
print("✅ Dtypes:\n", submission.dtypes)
print("✅ Nulls:\n", submission.isna().sum().sum())

✅ Shape: (116712, 207)
✅ Columns: ['row_id', '1139490', '1192948', '1194042', '126247', '1346504', '134933', '135045', '1462711', '1462737', '1564122', '21038', '21116', '21211', '22333', '22973', '22976', '24272', '24292', '24322', '41663', '41778', '41970', '42007', '42087', '42113', '46010', '47067', '476537', '476538', '48124', '50186', '517119', '523060', '528041', '52884', '548639', '555086', '555142', '566513', '64862', '65336', '65344', '65349', '65373', '65419', '65448', '65547', '65962', '66016', '66531', '66578', '66893', '67082', '67252', '714022', '715170', '787625', '81930', '868458', '963335', 'amakin1', 'amekes', 'ampkin1', 'anhing', 'babwar', 'bafibi1', 'banana', 'baymac', 'bbwduc', 'bicwre1', 'bkcdon', 'bkmtou1', 'blbgra1', 'blbwre1', 'blcant4', 'blchaw1', 'blcjay1', 'blctit1', 'blhpar1', 'blkvul', 'bobfly1', 'bobher1', 'brtpar1', 'bubcur1', 'bubwre1', 'bucmot3', 'bugtan', 'butsal1', 'cargra1', 'cattyr', 'chbant1', 'chfmac1', 'cinbec1', 'cocher1', 'cocwoo1', 'colara1'